# Practice Assignment 2
# Joining & Filtering Data

For this practice assignment some of the code has been written for you ahead of time.  Cells you just need to run are mark 'Run this cell'.  For each question, there are clear instructions in each cell. Follow those instructions and write the code after each block of:

YOUR CODE HERE

Please use the exact variable name if it is specified in the comment.

We’ll run a Python test script against your program to test whether each function implementation is correct.

In [1]:
%%capture
###########################################################
### EXECUTE THIS CELL BEFORE YOU TO TEST YOUR SOLUTIONS ###
###########################################################
"""
Loading the necessary python libraries
"""
import imp
import pandas as pd
import numpy as np
from nose.tools import assert_equal
from pandas.util.testing import assert_frame_equal, assert_series_equal
sol = imp.load_compiled("sol", "./.sol.py")

In [2]:
"""
First we need to load the dataset and preprocess the data. The below code comes from some of the work done in Practice Assignment 01.
"""

#Run this cell
restaurant_location_df = pd.read_csv("geoplaces2.csv")
restaurant_accept_df = pd.read_csv("chefmozaccepts.csv")
restaurant_cuisine_df = pd.read_csv("chefmozcuisine.csv")
rating_final_df = pd.read_csv("rating_final.csv")
user_profile_df = pd.read_csv("userprofile.csv")

restaurant_location_df = restaurant_location_df[["placeID", "name", "address", "city", "state", "country", "price", "alcohol"]]
restaurant_location_df = restaurant_location_df.apply(lambda x: x.astype(str).str.lower())
restaurant_location_df = restaurant_location_df.astype({'placeID':'int',"name":"string", "address":"string","city":"string", "state":"string", "country":"string", "price":"string", "alcohol": "string"})
restaurant_location_df = restaurant_location_df.replace('\?', np.nan, regex=True).dropna().reset_index(drop = True)
user_profile_df = user_profile_df.replace('\?', np.nan, regex=True)
user_profile_df = user_profile_df[['userID', 'smoker', 'drink_level', 'marital_status', 'birth_year']]

#Since The values in column 'smoker' are strings, we have to convert them to boolean values
user_profile_df['smoker'] = user_profile_df['smoker'].map({'true': True, 'false': False})
###

In [3]:
"""
Lets first examine the columns and datatypes for our "restaurant_location_df" dataframe.
"""

#Run this cell
restaurant_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   placeID  130 non-null    int64 
 1   name     130 non-null    string
 2   address  130 non-null    string
 3   city     130 non-null    string
 4   state    130 non-null    string
 5   country  130 non-null    string
 6   price    130 non-null    string
 7   alcohol  130 non-null    string
dtypes: int64(1), string(7)
memory usage: 8.2 KB


In [4]:
"""
Now let's display the "restaurant_location_df" dataframe.  As a reminder, we can use display() to display
dataframes as easy-to-read html tables.
"""

#Run this cell
display(restaurant_location_df)

,placeID,name,address,city,state,country,price,alcohol
0,134999,kiku cuernavaca,revolucion,cuernavaca,morelos,mexico,medium,no_alcohol_served
1,132825,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,low,no_alcohol_served
2,135106,el rinc�n de san francisco,universidad 169,san luis potosi,san luis potosi,mexico,medium,wine-beer
3,132667,little pizza emilio portes gil,calle emilio portes gil,victoria,tamaulipas,?,low,no_alcohol_served
4,132613,carnitas_mata,lic. emilio portes gil,victoria,tamaulipas,mexico,medium,no_alcohol_served
...,...,...,...,...,...,...,...,...
125,132866,chaires,ricardo b. anaya,san luis potosi,san luis potosi,mexico,medium,no_alcohol_served
126,135072,sushi itto,venustiano carranza 1809 c polanco,san luis potosi,slp,mexico,medium,no_alcohol_served
127,135109,paniroles,?,?,?,?,medium,wine-beer
128,135019,restaurant bar coty y pablo,paseo de las fuentes 24 pedregal de las fuentes,jiutepec,morelos,mexico,low,no_alcohol_served


In [5]:
"""
To quickly get an idea of the rows in all of the dataframes, we'll use .head() in the following 4 cells.
View the first 5 rows for "restaurant_accept_df".
"""

#Run this cell
restaurant_accept_df.head()

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards


In [6]:
"""
View the first 5 rows for "restaurant_cuisine_df".
"""

#Run this cell
restaurant_cuisine_df.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [7]:
"""
View the first 5 rows for "user_profile_df".
"""

#Run this cell
user_profile_df.head()

,userID,smoker,drink_level,marital_status,birth_year
0,U1001,False,abstemious,single,1989
1,U1002,False,abstemious,single,1990
2,U1003,False,social drinker,single,1989
3,U1004,False,abstemious,single,1940
4,U1005,False,abstemious,single,1992


In [8]:
"""
View the first 5 rows for "rating_final_df".
"""

#Run this cell
rating_final_df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [9]:
"""
Part 01

1. We want to see which restaurants can accept VISA payments.  In order to do this we need to merge
"restaurant_location_df" and "restaurant_accept_df".  Do this as a left join and store in a new dataframe called
"joined_df".  Examine both dataframes to identify the best columns to merge on.

2. Display the first 5 rows of "joined_df"

3. Create a new dataframe called "restaurant_visa_df" which contains only two columns from "joined_df":
'name' and 'Rpayment'.  Also filter the dataframe so that the 'Rpayment' column only contains those who paid
with 'VISA'. 

*** Please note: After performing the merge and filtering operations, ensure reset_index(drop=True) 
before submission to aviod Index-related Errors.

4. Display the first 10 rows of "restaurant_visa_df"
"""

# YOUR CODE HERE
joined_df = pd.merge(left=restaurant_location_df, right=restaurant_accept_df, how='left', on='placeID')

joined_df.head()

,placeID,name,address,city,state,country,price,alcohol,Rpayment
0,134999,kiku cuernavaca,revolucion,cuernavaca,morelos,mexico,medium,no_alcohol_served,NaN
1,132825,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,low,no_alcohol_served,cash
2,135106,el rinc�n de san francisco,universidad 169,san luis potosi,san luis potosi,mexico,medium,wine-beer,cash
3,135106,el rinc�n de san francisco,universidad 169,san luis potosi,san luis potosi,mexico,medium,wine-beer,VISA
4,135106,el rinc�n de san francisco,universidad 169,san luis potosi,san luis potosi,mexico,medium,wine-beer,MasterCard-Eurocard


In [10]:
restaurant_visa_df = joined_df[['name','Rpayment']]
restaurant_visa_df.head()

,name,Rpayment
0,kiku cuernavaca,NaN
1,puesto de tacos,cash
2,el rinc�n de san francisco,cash
3,el rinc�n de san francisco,VISA
4,el rinc�n de san francisco,MasterCard-Eurocard


In [11]:
restaurant_visa_df = restaurant_visa_df[restaurant_visa_df['Rpayment'] == "VISA"]

In [12]:
restaurant_visa_df.head(10)

,name,Rpayment
3,el rinc�n de san francisco,VISA
8,restaurant los compadres,VISA
12,shi ro ie,VISA
21,el angel restaurante,VISA
25,restaurante pueblo bonito,VISA
29,mcdonalds parque tangamanga,VISA
33,sirlone,VISA
39,restaurant el muladar de calzada,VISA
43,la posada del virrey,VISA
48,restaurant and bar and clothesline carlos n ch...,VISA


In [13]:
restaurant_visa_df.reset_index(inplace=True, drop=True)

In [14]:
restaurant_visa_df

,name,Rpayment
0,el rinc�n de san francisco,VISA
1,restaurant los compadres,VISA
2,shi ro ie,VISA
3,el angel restaurante,VISA
4,restaurante pueblo bonito,VISA
5,mcdonalds parque tangamanga,VISA
6,sirlone,VISA
7,restaurant el muladar de calzada,VISA
8,la posada del virrey,VISA
9,restaurant and bar and clothesline carlos n ch...,VISA


In [15]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(joined_df, sol.joined_df)
print("test joined_df correct")
assert_frame_equal(restaurant_visa_df, sol.restaurant_visa_df)
print("test restaurant_visa_df correct")


test joined_df correct
test restaurant_visa_df correct


In [ ]:
"""
Part 02

1. First lets merge "restaurant_location_df" and "rating_final_df" with a left join.  Store the result in a new
dataframe called "restaurant_mean_rating_df".  Examine both dataframes to identify the best columns to merge on.

2. "restaurant_mean_rating_df" should only keep 3 columns after it has been merged: 'name', 'food_rating',
and 'service_rating'.

3. Filter "restaurant_mean_rating_df" by its 'name' column, for restaurants with the name
'restaurant bar coty y pablo'

4. Display the updated "restaurant_mean_rating_df".  Hint: If everything was completed correctly, you should
have 3 columns and 6 rows.

5. Calculate the average (mean) of the 'food_rating' column and store in a new variable called
"mean_food_rating_pablo"

6. Calculate the average (mean) of the 'service_rating' column and store in a new variable called
"mean_ser_rating_pablo"

7. Print both of the means you just calculated
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################


assert_frame_equal(restaurant_mean_rating_df, sol.restaurant_mean_rating_df)
print("test restaurant_mean_rating_df correct")
assert_equal(mean_food_rating_pablo, sol.mean_food_rating_pablo)
print("test mean_food_rating_pablo correct")
assert_equal(mean_ser_rating_pablo, sol.mean_ser_rating_pablo)
print("test mean_ser_rating_pablo correct")



In [ ]:
"""
Part 03

For this part we want to see the average rating of all restaurants by "single smokers" and the average
rating of all restaurants by "married non-smokers".

1. Merge "rating_final_df" (as the left df) and "user_profile_df" (as the right df) using an inner join, and name the resulting dataframe
"join_rating_df".  Examine both dataframes to identify the best columns to merge on.

2. Create a new dataframe called "single_smoke_df", which has 3 columns: 'smoker', 'marital_status',
and 'rating'. This should be a slice of the join_rating_df your just created.

3. Filter "single_smoke_df" where the 'smoker' column is True, and the 'marital_status' column is 'single'

4. Calcualate the average (mean) of the ratings in "single_smoke_df" and store in a variable called
"single_smoke_rating"

5. Repeat steps 2 - 4 for married non-smokers and name the dataframe "married_nonsmoke_df" and the final average
variable "married_nonsmoke_rating".  Hint: We are asking for non-smokers now, so adjust your filters as needed.

6. Print out the results for your variables rounded to 2 decimal places ('single_smoke_rating' and
'married_nonsmoke_rating')
"""

# YOUR CODE HERE


In [ ]:
##########################
### TEST YOUR SOLUTION ###
##########################

assert_frame_equal(join_rating_df, sol.join_rating_df)
print("test join_rating_df correct")

assert_frame_equal(single_smoke_df, sol.single_smoke_df)
print("test single_smoke_df correct")

assert_equal(single_smoke_rating, sol.single_smoke_rating)
print("test single_smoke_rating correct")

assert_frame_equal(married_nonsmoke_df, sol.married_nonsmoke_df)
print("test married_nonsmoke_df correct")
assert_equal(married_nonsmoke_rating, sol.married_nonsmoke_rating)
print("test married_nonsmoke_rating correct")
